In [1]:
import os
import numpy as np
import rasterio as rio
from copy import deepcopy

In [2]:
dataset = r"/home/abetatos/Escritorio/TFM_Dataset"

outp = "4.DatasetPost"
inp = "3.Basedataset"

inpath = os.path.join(dataset, inp)
outpath = os.path.join(dataset, outp)

In [3]:
FEATURES =  os.listdir(inpath)
LOCATIONS = os.listdir(os.path.join(inpath, "HS"))

In [4]:
def get_array(inpath_file): 
    with rio.open(inpath_file, "r") as src: 
        array = src.read()
    return array

def get_arrays(feature):
    arrays = []
    for location in LOCATIONS:
        inpath_file = os.path.join(inpath, feature, location, feature + ".tif")
        arrays.append(get_array(inpath_file))
    return arrays

def get_lims(feature):
    arrays = get_arrays(feature)
    maximum = np.array([a.max() for a in arrays]).max()
    for a in arrays:
        a[a<=-9999.] = np.inf
    minimum = np.array([a.min() for a in arrays]).min()
    return maximum, minimum

In [5]:
def rescale(array, minimum, maximum): 
    mask = array==-9999.
    array = (array - minimum)/(maximum - minimum)
    array[mask] = -9999.
    return array

In [6]:
for feature in FEATURES:
    maximum, minimum = get_lims(feature)
    print(feature, maximum, minimum)
    for location in LOCATIONS:
        inpath_file = os.path.join(inpath, feature, location, feature + ".tif")
        outpath_file = os.path.join(outpath, feature, location, feature + ".tif")

        with rio.open(inpath_file, "r") as src:
            array = src.read()[0]
            profile = src.profile
        
        os.makedirs(os.path.dirname(outpath_file), exist_ok=True)
        with rio.open(outpath_file, "w", **profile) as src: 

            if feature != "HS":
                src.write(rescale(array, minimum, maximum), 1)
            else:
                src.write(array, 1)

DEM 3408.71 959.74744
SCE 0.9726027 0.0
FFSnow 180.0 0.0
DEMSNOW 6.17763 -0.1650578
TPI 1.0 -1.0
Slope 33.00405 0.0
TPIWGW 8201.83 -9337.114
Aspect 359.99948 0.0
HS 26.533447 -18.737549


In [43]:
array.min()

-9999.0